In [1]:
import torch
from torch import nn
from math import inf
from tqdm import tqdm
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.transforms import ToTensor

In [2]:
fashion_train = datasets.FashionMNIST(root="./datasets/mnist", download=True, train=True)#, transform=ToTensor())
#fashion_test = datasets.FashionMNIST(root="./datasets/mnist", download=True, train=False, transform=ToTensor())

In [3]:
fashion_train

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./datasets/mnist
    Split: Train

In [4]:
print(fashion_train[0])

(<PIL.Image.Image image mode=L size=28x28 at 0x260B6A17B20>, 9)


In [5]:
fashion_train[0][0]

In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.2860,), (0.3530,))
])

In [7]:
fashion_train = datasets.FashionMNIST(root="./datasets/mnist", download=True, train=True, transform=transform)
fashion_test = datasets.FashionMNIST(root="./datasets/mnist", download=True, train=False, transform=transform)

In [8]:
train_loader = DataLoader(fashion_train, batch_size=512, shuffle=True)
test_loader = DataLoader(fashion_test, batch_size=512, shuffle=True)

In [9]:
len(fashion_train.classes)

10

In [10]:
fashion_train.classes

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [40]:
model = nn.Sequential(
    nn.Conv2d(1, 3, kernel_size=(3, 3), padding=1, padding_mode="reflect"),
    nn.BatchNorm2d(3), 
    nn.ReLU(),
    nn.MaxPool2d(3, 3),
    nn.Conv2d(3, 6, kernel_size=(3, 3), padding=1, padding_mode="reflect"),
    nn.BatchNorm2d(6),
    nn.ReLU(),
    nn.MaxPool2d(3, 3),
    nn.Conv2d(6, 12, kernel_size=(3, 3), padding=1, padding_mode="reflect"),
    nn.BatchNorm2d(12),
    nn.Flatten(),
    nn.Linear(108, 16),
    nn.Dropout(0.2),
    nn.Linear(16, 10)
)

In [41]:
model

Sequential(
  (0): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (5): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (9): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=108, out_features=16, bias=True)
  (12): Dropout(p=0.2, inplace=False)
  (13): Linear(in_features=16, out_features=10, bias=True)
)

In [42]:
image = fashion_train[0][0]
outputs = model(image.reshape(1, 1, 28, 28))
print(outputs.shape)

torch.Size([1, 10])


In [43]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [44]:
for x, y in tqdm(train_loader):
    print(x.shape, y.shape)
    break

  0%|                              | 0/118 [00:00<?, ?it/s]

torch.Size([512, 1, 28, 28]) torch.Size([512])


In [45]:
model.to("cuda")

Sequential(
  (0): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (5): BatchNorm2d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(6, 12, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), padding_mode=reflect)
  (9): BatchNorm2d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): Flatten(start_dim=1, end_dim=-1)
  (11): Linear(in_features=108, out_features=16, bias=True)
  (12): Dropout(p=0.2, inplace=False)
  (13): Linear(in_features=16, out_features=10, bias=True)
)

In [46]:
early_stopping = 0
old_loss = inf
for i in range(100):
    print(f"[] Epochs: {i+1}")
    total_loss = 0
    model.train()
    for x, y in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(x.to("cuda"))
        y = F.one_hot(y, num_classes=10).type(torch.float32).to("cuda")
        loss = loss_fn(outputs, y)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    loss_value = total_loss
    if loss_value < old_loss:
        old_loss = loss_value
        early_stopping = 0
    else:
        early_stopping += 1
    if early_stopping > 2:
        print("Early stopping triggered")
        break
    print(total_loss)

[] Epochs: 1


100%|████████████████████| 118/118 [00:13<00:00,  8.43it/s]


200.20375418663025
[] Epochs: 2


100%|████████████████████| 118/118 [00:13<00:00,  8.54it/s]


124.34145128726959
[] Epochs: 3


...

48.97142136096954
[] Epochs: 64


100%|████████████████████| 118/118 [00:13<00:00,  8.43it/s]


49.04601365327835
[] Epochs: 65


100%|████████████████████| 118/118 [00:13<00:00,  8.56it/s]

Early stopping triggered


In [47]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0    
    test_loss = 0
    for x, y in tqdm(test_loader):
        x = x.to("cuda")
        y = y.to("cuda")    
        outputs = model(x)
        preds = outputs.argmax(dim=1)    
        correct += (preds == y).sum().item()    
        loss = loss_fn(outputs, y)
        total += y.size(0)   
        test_loss += loss.item() * y.size(0)
    accuracy = correct / total
    loss = test_loss / total
    print(f"Accuracy: {accuracy}, Test loss: {loss}")

100%|██████████████████████| 20/20 [00:02<00:00,  8.80it/s]

Accuracy: 0.8527, Test loss: 0.4058666379928589
